# Predicting if sequence is linear or not
this is a tutorial on how to implement a Dynamcic LSTM.

In [2]:
import numpy as np

In [3]:
import tensorflow as tf

# Creating linear and random sequences
class 0 == linear sequences [10,11,12...]
class 1 == random sequences [10,20,1,...]

In [4]:
print(tf.__version__)

1.12.0


In [24]:
import random
n_samples = 1500
max_seq_len = 10
min_seq_len = 5
max_val = 200
data = []
labels = []

for i in range(n_samples):
    len1=random.randint(min_seq_len,max_seq_len)
    if(random.random() < .5) :
        start = random.randint(0,max_val-len1)
        s = [float(i) for i in range(start,start+len1)]
#         s=np.asarray(s)
        data.append(s)
        labels.append([1,0])
    else :
        s = [float(random.randint(0,max_val)) for i in range(len1)]
#         s = np.asarray(s)
        data.append(s)
        labels.append([0,1])

In [25]:
# data = np.asarray(data)
labels = np.asarray(labels)
print(data[5:10])
print(labels[5:10])

[[11.0, 174.0, 160.0, 196.0, 109.0], [29.0, 115.0, 31.0, 79.0, 199.0, 185.0, 126.0], [38.0, 60.0, 117.0, 113.0, 145.0], [94.0, 131.0, 103.0, 157.0, 161.0, 118.0, 199.0, 128.0, 154.0], [111.0, 87.0, 160.0, 154.0, 39.0, 128.0, 48.0, 149.0]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [26]:
data.shape

AttributeError: 'list' object has no attribute 'shape'

# Model declaration

In [39]:
tf.reset_default_graph()
batch_size = 50
display_step = 100

num_hidden = 64
num_classes = 2

num_layers = 2

learning_rate = tf.placeholder(tf.float32,name='lr_rate')
keep_prob = tf.placeholder(tf.float32,name='keep_prob')

seqlen = tf.placeholder(tf.int32,[None],name='sequence_length')

sequence_maximum_length = tf.reduce_max(seqlen)

x = tf.placeholder("float",[None,None,1])
y = tf.placeholder("float",[None,num_classes])

# cell = tf.contrib.rnn.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(rnn_size), keep_prob)

# x = tf.unstack(x,sequence_maximum_length,1)


stacked_cells = tf.contrib.rnn.MultiRNNCell(
    [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(num_hidden), keep_prob) for _ in range(num_layers)])

##outputs obtained here have max_length of batch but computation is done only until
init_state = stacked_cells.zero_state(batch_size, tf.float32)
outputs, last_states = tf.nn.dynamic_rnn(stacked_cells,x,initial_state=init_state,sequence_length=seqlen,dtype=tf.float32)


#softmax layer for output

W = tf.Variable(tf.random_normal([num_hidden,num_classes]))

b = tf.Variable(tf.random_normal([num_classes]))


##We need to get outputs at which dynamic rnn computation is stopped

outputs = tf.stack(outputs)

# Outputs of _dynamic_rnn_loop are always shaped [time, batch, depth].
#so converting it to [batch,time,depth]
outputs = tf.transpose(outputs, [1, 0, 2])

index = tf.range(0,batch_size) * sequence_maximum_length + (seqlen -1)

#inputs to softmax layers [batch_size*num_hidden]
outputs = tf.gather(tf.reshape(outputs,[-1,num_hidden]),index)

#logits 
preds = tf.matmul(outputs,W)+b

#Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#Evaluate model
correct_pred = tf.equal(tf.argmax(preds,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#Initialize the variables
init = tf.global_variables_initializer()

In [54]:
o = np.arange(30).reshape((2,3,5))
print(o)
xx = tf.unstack(o)
o = tf.stack(o)
# o = tf.transpose(o,[1,0,2])
seqlen = np.array([3,1])
# print(seqlen-1)
seq_max_len = 3
index = seqlen - 1
index = tf.range(0,2)*3 + (seqlen - 1 )
o = tf.reshape(o,[-1,5])
o = tf.gather(tf.reshape(o,[-1,5]),index)
with tf.Session() as sess:
    print(sess.run(index))
    print(sess.run(xx))
#tf.transpose(o,[1,0,2])

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]]

 [[15 16 17 18 19]
  [20 21 22 23 24]
  [25 26 27 28 29]]]
[2 3]
[array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]]), array([[15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29]])]


In [31]:
# data = np.stack(data)
data_train = data[:1300]
# data_train = data_train[:, :, np.newaxis]
label_train =labels[:1300]
data_test = data[1300:]
label_test = labels[:1300]
# for src in data_train:
#     print(src.shape[0])
# print(data_train.shape)
# print(label_train.shape[0])
print(len(data))

1500


In [36]:
def get_batches(sources,labels,batch_size):
    """Batch targets, sources, and the lengths of their sentences together"""
    # print(len(sources)//batch_size)
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        seq_length = []
        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = labels[start_i:start_i + batch_size]
        
        for source in sources_batch:
            seq_length.append(len(source))
        
        yield sources_batch,targets_batch,seq_length

In [37]:
num_epochs=10
keep_probability=0.5
learning_rate=0.01
with tf.Session() as sess:
    sess.run(init)
    for epoch_i in range(1,num_epochs+1):
        for batch_i,(source_batch, labels_batch,source_lengths) in enumerate(get_batches(data_train,label_train,batch_size)):
            print('epoch :: {}, batch :: {} '.format(epoch_i,batch_i))
            sess.run(optimizer,feed_dict={x:source_batch,y:labels_batch,seqlen:source_lengths,learning_rate:learning_rate,keep_prob:keep_probability})
            if step % display_step == 0 or step == 1:
                acc,loss = sess.run([accuracy,cost],feed_dict={x:source_batch,y:labels_batch,seqlen:source_lengths})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

epoch :: 1, batch :: 0 


ValueError: setting an array element with a sequence.